# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df1 = pd.read_csv(file_path)

# Preview the data
df1.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
# feature selection
selected_features=list(df1.columns)
selected_features.remove('loan_status')

In [6]:
df=pd.get_dummies(df1[selected_features])
df=pd.concat([df, df1['loan_status']], axis=1)
df.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,loan_status
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,0,1,0,1,1,0,1,1,low_risk
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,0,1,0,1,1,0,1,1,low_risk
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,0,1,0,1,1,0,1,1,low_risk
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,0,1,0,1,1,0,1,1,low_risk
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,0,1,0,1,1,0,1,1,low_risk


In [7]:
df.columns

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_in

In [8]:
df['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

## Split the Data into Training and Testing

In [9]:
# Create our features #inputs/columns
X = df.copy()
X.drop(columns=["loan_status"], axis=1, inplace=True)
X.head()

# Create our target
y = df["loan_status"]

In [10]:
# Check the balance of our target values
df["loan_status"].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

# Needed for decision tree visualization
import pydotplus
from IPython.display import Image

In [12]:
# Split the X and y into X_train, X_test, y_train, y_test #default-splits into 75% TRAIN and 25% TEST
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(51612, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [13]:
# Create the StandardScaler instance      #puts the data into the same units-using std from mean(z-score)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [14]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)



In [15]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [16]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
bdf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
bdf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [17]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = bdf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8192848020434227

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   62,    25],
       [ 1268, 15850]], dtype=int64)

In [19]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.05      0.71      0.93      0.09      0.81      0.65        87
   low_risk       1.00      0.93      0.71      0.96      0.81      0.67     17118

avg / total       0.99      0.92      0.71      0.96      0.81      0.67     17205



In [20]:
# List the features sorted in descending order by feature importance

sorted(list(zip(bdf.feature_importances_, X_train.columns)), reverse=True)

#X_test.sort_values(by='total_rec_prncp', ascending=False)

[(0.06928717515721282, 'total_rec_prncp'),
 (0.06195341602973416, 'total_rec_int'),
 (0.05999716770508804, 'total_pymnt'),
 (0.05944929931774028, 'total_pymnt_inv'),
 (0.05780080255994967, 'last_pymnt_amnt'),
 (0.02708701665429169, 'int_rate'),
 (0.019362938516910073, 'issue_d_Jan-2019'),
 (0.01765723264865733, 'dti'),
 (0.01635551128868266, 'installment'),
 (0.0162869588930354, 'out_prncp'),
 (0.016075098739934283, 'annual_inc'),
 (0.015822865990250717, 'revol_bal'),
 (0.01573225080912679, 'out_prncp_inv'),
 (0.015462048804453772, 'max_bal_bc'),
 (0.01511218923248116, 'bc_util'),
 (0.014818248548877595, 'mths_since_recent_inq'),
 (0.014736763773410759, 'mo_sin_old_il_acct'),
 (0.014668152866939446, 'avg_cur_bal'),
 (0.01454052198931068, 'il_util'),
 (0.014405428481318226, 'bc_open_to_buy'),
 (0.014273402855444749, 'mo_sin_old_rev_tl_op'),
 (0.01408832804623083, 'issue_d_Mar-2019'),
 (0.01407588836833831, 'total_bc_limit'),
 (0.013462077489263542, 'tot_hi_cred_lim'),
 (0.01341647541458

### Easy Ensemble Classifier

In [21]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
edf = EasyEnsembleClassifier(n_estimators=100, random_state=1)
edf.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [22]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = edf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.925427358175101

In [23]:
sorted(list(zip(edf.feature_importances_, X_train.columns)), reverse=True)

AttributeError: 'EasyEnsembleClassifier' object has no attribute 'feature_importances_'

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE. 

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.

In [ ]:
1) Easy Ensemble
2) Easy Ensemble
3) Easy Ensemble
4) 